In [ ]:
import numpy as np
import matplotlib.pyplot as plt

import torch
import torchvision
import torchvision.transforms as transforms
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

from tqdm import tqdm

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cuda:0


In [ ]:
import os
from google.colab import drive
drive.mount('/content/drive')
checkpoints = '/content/drive/MyDrive/colab_files/DenseNet/'
if not os.path.exists(checkpoints):
    os.makedirs(checkpoints)

In [ ]:
os.chdir('drive/MyDrive') # Need to change directory to the google drive before downloading datasaet
if not os.path.exists('birds21sp'):
    !mkdir birds21sp
    os.chdir('birds21sp')
    !wget https://pjreddie.com/media/files/birds/train.tar
    !wget https://pjreddie.com/media/files/birds/test.tar
    !wget https://pjreddie.com/media/files/birds/names.txt
    !tar xf train.tar
    !tar xf test.tar
    !mkdir testing
    !mv test testing
    os.chdir('..')

In [ ]:
def get_bird_data(augmentation=0, validation = False):
    transform_train = transforms.Compose([
        transforms.Resize(256),
        transforms.RandomCrop(256, padding=8, padding_mode='edge'), # Take 256x256 crops from padded images
        transforms.RandomHorizontalFlip(),    # 50% of time flip image along y-axis
        transforms.ToTensor(),
        # transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ])
    
    transform_test = transforms.Compose([
        transforms.Resize(256),
        transforms.ToTensor(),
    ])
    trainset = torchvision.datasets.ImageFolder(root='birds21sp/train', transform=transform_train)

    if validation:      
      # Make a validation set
      validation_len = len(trainset) // 10
      train_len = len(trainset) - validation_len
      subsets = torch.utils.data.random_split(trainset, [train_len, validation_len])

      trainloader = torch.utils.data.DataLoader(subsets[0], batch_size=16,
                                                shuffle=True, num_workers=2)

      valloader = torch.utils.data.DataLoader(subsets[1], batch_size=16,
                                                shuffle=True, num_workers=2)
    else:
      trainloader = torch.utils.data.DataLoader(trainset, batch_size=8,
                                                shuffle=True, num_workers=2)
      valloader = None
      validation_len = 0

    testset = torchvision.datasets.ImageFolder(root='birds21sp/testing', transform=transform_test)
    testloader = torch.utils.data.DataLoader(testset, batch_size=1, shuffle=False, num_workers=2)
    classes = open("birds21sp/names.txt").read().strip().split("\n")
    class_to_idx = trainset.class_to_idx
    idx_to_class = {int(v): int(k) for k, v in class_to_idx.items()}
    idx_to_name = {k: classes[v] for k,v in idx_to_class.items()}
    if validation:
      return {'train': trainloader, 'val' : valloader, 'val_len': validation_len, 'test': testloader, 'to_class': idx_to_class, 'to_name':idx_to_name}
    return {'train': trainloader, 'test': testloader, 'to_class': idx_to_class, 'to_name':idx_to_name}

data = get_bird_data()

In [ ]:
# Check Images
dataiter = iter(data['train'])
images, labels = dataiter.next()
images = images[:8]
print(images.size())

def imshow(img):
    npimg = img.numpy()
    plt.imshow(np.transpose(npimg, (1, 2, 0)))
    plt.show()

# show images
imshow(torchvision.utils.make_grid(images))
# print labels
print("Labels:" + ', '.join('%9s' % data['to_name'][labels[j].item()] for j in range(8)))

In [ ]:
def train(net, dataloader, epochs=1, start_epoch=0, lr=0.01, momentum=0.9, decay=0.0005, 
          verbose=1, print_every=10, state=None, schedule={}, checkpoint_path=None):
    net.to(device)
    net.train()
    losses = []
    criterion = nn.CrossEntropyLoss()
    # optimizer = optim.Adam(net.parameters, lr=lr, weight_decay=decay)
    optimizer = optim.SGD(net.parameters(), lr=lr, momentum=momentum, weight_decay=decay)

    # Load previous training state
    if state:
        net.load_state_dict(state['net'])
        optimizer.load_state_dict(state['optimizer'])
        start_epoch = state['epoch']
        losses = state['losses']

    # Fast forward lr schedule through already trained epochs
    for epoch in range(start_epoch):
        if epoch in schedule:
            print ("Learning rate: %f"% schedule[epoch])
            for g in optimizer.param_groups:
                g['lr'] = schedule[epoch]

    for epoch in tqdm(range(start_epoch, epochs)):
        sum_loss = 0.0

        # Update learning rate when scheduled
        if epoch in schedule:
            print ("Learning rate: %f"% schedule[epoch])
            for g in optimizer.param_groups:
                g['lr'] = schedule[epoch]

        for i, batch in enumerate(dataloader, 0):
            inputs, labels = batch[0].to(device), batch[1].to(device)

            optimizer.zero_grad()

            outputs = net(inputs)
            loss = criterion(outputs, labels)
            loss.backward()  # autograd magic, computes all the partial derivatives
            optimizer.step() # takes a step in gradient direction

            losses.append(loss.item())
            sum_loss += loss.item()

            if i % print_every == print_every-1:    # print every 10 mini-batches
                if verbose:
                  print('[%d, %5d] loss: %.3f' % (epoch, i + 1, sum_loss / print_every))
                sum_loss = 0.0
        if checkpoint_path:
            state = {'epoch': epoch+1, 'net': net.state_dict(), 'optimizer': optimizer.state_dict(), 'losses': losses}
            torch.save(state, checkpoint_path + 'checkpoint-%d.pkl'%(epoch+1))
    return losses

In [ ]:
def getResNet():
  model = torch.hub.load('pytorch/vision:v0.9.0', 'resnet50', pretrained=True)
  model.fc = nn.Linear(2048, 555) # This will reinitialize the layer as well
  return model

def getResNext():
  model = torch.hub.load('pytorch/vision:v0.9.0', 'resnext101_32x8d', pretrained=True)
  model.fc = nn.Linear(2048, 555) # This will reinitialize the layer as well
  return model

def getDenseNet():
  model = torch.hub.load('pytorch/vision:v0.9.0', 'densenet201', pretrained=True)
  model.classifier = nn.Linear(1920, 555) # This will reinitialize the layer as well
  return model

In [ ]:
# Training from checkpoints, 
model = getDenseNet()
state = torch.load(checkpoints + 'checkpoint-13.pkl')
# state = None # If we don't want to train from checkpoints
 losses = train(model, data['train'], epochs=15, schedule={0:.01, 7:.001, 13: .0001}, lr=.01, print_every=10, checkpoint_path=checkpoints, state=state)

In [ ]:
def smooth(x, size):
  return np.convolve(x, np.ones(size)/size, mode='valid')

In [ ]:
state = torch.load(checkpoints + 'checkpoint-13.pkl', map_location=torch.device('cpu'))
plt.plot(smooth(state['losses'], 200), label = '13 epoch')
state = torch.load(checkpoints + 'checkpoint-6.pkl', map_location=torch.device('cpu'))
plt.plot(smooth(state['losses'], 200), label = '6 epoch')
plt.legend()

In [ ]:
def count_accurate(output, labels):
  classification = torch.argmax(output, 1)
  return torch.sum(classification == labels).item()
  
# Check accuracy with validation set
def getValidationAcc(model, data):
  model.to(device)
  model.eval()
  loss_func = nn.CrossEntropyLoss()
  accurate = 0
  epoch_loss = 0

  for i, in_out in tqdm(enumerate(data['val'], 0)):
    input, labels = in_out
    input = input.to(device)
    labels = labels.to(device)

    with torch.no_grad():
      output = model(input)
      loss = loss_func(output, labels)
    
    epoch_loss += loss.item()
    accurate += count_accurate(output, labels)

  accurate /= data['val_len']
  print("Validation loss: " + str(epoch_loss))
  print("Validation acc: " + str(accurate))

In [ ]:
def predict(net, dataloader, ofname):
    out = open(ofname, 'w')
    out.write("path,class\n")
    net.to(device)
    net.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for i, (images, labels) in enumerate(dataloader, 0):
            if i%100 == 0:
                print(i)
            images, labels = images.to(device), labels.to(device)
            outputs = net(images)
            _, predicted = torch.max(outputs.data, 1)
            fname, _ = dataloader.dataset.samples[i]
            out.write("test/{},{}\n".format(fname.split('/')[-1], data['to_class'][predicted.item()]))
    out.close()

In [ ]:
# Load model from checkpoint
model = torch.hub.load('pytorch/vision:v0.9.0', 'resnext101_32x8d', pretrained=True)
model.fc = nn.Linear(2048, 555) # This will reinitialize the layer as well
state = torch.load(checkpoints + 'checkpoint-10.pkl')
model.load_state_dict(state['net'])

# getValidationAcc(model, data)
# predict(model, data['test'], checkpoints + "d.csv")